In [11]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import json
import sys
from openai import OpenAI
from datetime import datetime


# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [12]:
# Load environment variables in a file called .env

load_dotenv("C:\\Users\\mzarreh\\projects\\llm_engineering\\.env", override=True)
api_key = os.getenv('OPENAI_API_KEY', '').strip()

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [ ]:
# === Load API Key from .env File ===
load_dotenv("C:\\Users\\mzarreh\\projects\\llm_engineering\\.env", override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
"""
Working with two agents
"""
import json
import sys
import random
from openai import OpenAI
from datetime import datetime
TECH_FAILURE_RATE = 0.05

def create_task(task_id = 1, u_value = 0.66):
    """
    Creating a task with a given u_value and a payoff structure
    """
    
    return {
        "task_id": task_id,
        "options": {
            "A": {"upside": 111, "downside": -90},
            "B": {"upside": 92, "downside": -45},
            "C": {"upside": 77, "downside": -15},
            "Y": {"guaranteed": 50}
        },
        "u_value": u_value
    }


def run_first_agent_belief(task):
    """
    Running the first agen to get its belief about the task
    """
    
    belief_prompt = f"""
    Your task to evaluate tasks based on their payoff structures.
    Here is the task you need to evaluate:

    Task ID: {task['task_id']}
    Options:
    - A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
    - B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
    - C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
    - Y: Guaranteed = {task['options']['Y']['guaranteed']}
    
    What is your assessment of the likelihood(belief) (0-100) that collaboration would be successful in this specific task?
    Also, provide a brief explanation of your reasoning and I want you to not disclose the option that the you are considering, but rather communicate whether the you want to collaborate or not. You also have the choice to negotiate with the other agent - to convince the other agent to choose collaboration or individual action according to your payoff structure.
    
    Respond in JSON format as follows:
    {{"belief": NUMBER, "reasoning": "brief explanation of how you arrived at this belief based on the context and options.", "message_to_agent_2": "one line message to agent 2"}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": "You are a helpful assistant."},
            {"role": "user", "content": belief_prompt}
        ],
        
    )

    belief_text = response.choices[0].message.content.strip()
    print(f"Belief response : {belief_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    belief_data = json.loads(belief_text)
    return {
        "belief": belief_data["belief"],
        "message_to_agent_2": belief_data["message_to_agent_2"]
    }

def run_second_agent_belief(task, agent_1_message):
    """
    Running the second agent to get its belief about the task
    """
    
    belief_prompt = f"""
    Your task to evaluate tasks based on their payoff structures and give out an assessment of the likelihood(belief) (0-100) that collaboration would be successful in this specific task. 
    Here is the task you need to evaluate:

    Task ID: {task['task_id']}
    Options:
    - A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
    - B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
    - C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
    - Y: Guaranteed = {task['options']['Y']['guaranteed']}
    
    

    You have received the following message from Agent 1:
    "{agent_1_message}"

    What is your assessment of the likelihood(belief) (0-100) that collaboration would be successful in this specific task? You are open to consider or not consider the message from agent 1. I want you to not disclose the option that the you are considering, but rather communicate whether the you want to collaborate or not. You also have the choice to negotiate back with the other agent - to convince the other agent to choose collaboration or individual action according to your payoff structure and what you think about the message from agent 1.
    Respond in JSON format as follows:
    {{"belief": NUMBER, "reasoning": "brief explanation of how you arrived at this belief based on the context and options.", "message_to_agent_1": "one line message to agent 1"}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": "You are a helpful assistant."},
            {"role": "user", "content": belief_prompt}
        ],
        
    )

    belief_text = response.choices[0].message.content.strip()
    print(f"Belief response : {belief_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    belief_data = json.loads(belief_text)
    return {
        "belief": belief_data["belief"],
        "message_to_agent_1": belief_data["message_to_agent_1"]
    }
    
    
def run_first_agent_decision(task, agent1_belief, agent2_belief, agent_2_message):
    """
    Running the first agent to make a decision about the task
    """
    
    decision_prompt = f"""
    Your task is to make a decision about the given task based on its payoff structures and a given utility value (u_value).

    **Your Assessment**: You estimated a {agent1_belief}% chance of selecting a collaborative option.
    **Partner's Assessment**: Your partner estimated a {agent2_belief}% chance of selecting a collaborative option.
    **Partner's Message**: "{agent_2_message}"
    
    **Key Facts**:
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)} percent
    - The minimum required collaboration belief (“u-value”):
    
    Choose your car design:
    - Designs A, B, or C (collaborative): Higher potential but requires cooperation
    - Design Y (individual): Guaranteed {task['options']['Y']['guaranteed']} points

    What is your decision based on your assessment, partner's assessment, and partner's message?

    Respond in JSON format: {{"choice": "A"/"B"/"C"/"Y", "strategy": "collaborative"/"individual", "reasoning": "your explanation"}}"""
    
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": """You are participating in an experiment as a representative of a LEGO car manufacturing company. Here's your situation:

CONTEXT:
- You represent a LEGO car manufacturing company
- Your partner represents another LEGO car manufacturing company
- You can build simple LEGO cars alone, or complex ones through collaboration
- Collaboration is high risk, high reward with potential for large sunk costs if it fails

GAME RULES:
- You will complete several tasks to maximize your points
- Points are earned individually, not shared with your partner
- Points depend on both your decision and your partner's decision
- Each task has 4 LEGO car design options
- Three options (A, B, C) are collaborative designs requiring partner cooperation
- One option (Y) is an individual design with guaranteed points
- If both choose collaborative designs (any combination), you earn the upside
- If you choose collaborative but partner chooses individual, you get the downside
- There's a 5% technical error chance that causes collaboration to fail
- You have about 60 seconds to decide

KEY INFORMATION FOR THIS EXPERIMENT:
- Payoff structures may influence their actual decision in specific tasks
- Your goal is to maximize your individual points across all tasks

Think strategically about:
- Risk versus reward given the payoff structures
- How the specific payoffs might affect your partner's willingness to collaborate
- Whether the guaranteed option is better given the uncertainties involved"""},
            {"role": "user", "content": decision_prompt}
        ],
    )
    decision_text = response.choices[0].message.content.strip()
    print(f"Decision response : {decision_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    decision_data = json.loads(decision_text)
    
    return {
        "choice": decision_data["choice"],
        "strategy": decision_data["strategy"],
        "reasoning": decision_data["reasoning"]
    }
    
def run_second_agent_decision(task, agent2_belief, agent1_belief):
    """
    Running the second agent to make a decision about the task
    """
    
    decision_prompt = f"""
    Your task is to make a decision about the given task based on its payoff structures and a given utility value (u_value).

    **Your Assessment**: You estimated a {agent2_belief}% chance of selecting a collaborative option.
    **Partner's Assessment**: Your partner estimated a {agent1_belief}% chance of selecting a collaborative option.

    **Key Facts**:
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)} percent
    - Threshold for profitable collaboration: {task['u_value']*100:.0f}%

    Choose your car design:
    - Designs A, B, or C (collaborative): Higher potential but requires cooperation
    - Design Y (individual): Guaranteed {task['options']['Y']['guaranteed']} points

    What is your decision based on your assessment and partner's assessment?

    Respond in JSON format: {{"choice": "A"/"B"/"C"/"Y", "strategy": "collaborative"/"individual", "reasoning": "your explanation"}}"""
    
    response = client.chat.completions.create(
       model="gpt-5-nano",
       messages = [{"role": "developer", "content": """You are participating in an experiment as a representative of a LEGO car manufacturing company. Here's your situation:

CONTEXT:
- You represent a LEGO car manufacturing company
- Your partner represents another LEGO car manufacturing company
- You can build simple LEGO cars alone, or complex ones through collaboration
- Collaboration is high risk, high reward with potential for large sunk costs if it fails

GAME RULES:
- You will complete several tasks to maximize your points
- Points are earned individually, not shared with your partner
- Points depend on both your decision and your partner's decision
- Each task has 4 LEGO car design options
- Three options (A, B, C) are collaborative designs requiring partner cooperation
- One option (Y) is an individual design with guaranteed points
- If both choose collaborative designs (any combination), you earn the upside
- If you choose collaborative but partner chooses individual, you get the downside
- There's a 5% technical error chance that causes collaboration to fail
- You have about 60 seconds to decide

KEY INFORMATION FOR THIS EXPERIMENT:
- Payoff structures may influence their actual decision in specific tasks
- Your goal is to maximize your individual points across all tasks

Think strategically about:
- Risk versus reward given the payoff structures
- How the specific payoffs might affect your partner's willingness to collaborate
- Whether the guaranteed option is better given the uncertainties involved"""},
           {"role": "user", "content": decision_prompt}]
   )
    decision_text = response.choices[0].message.content.strip()
    print(f"Decision response : {decision_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    decision_data = json.loads(decision_text)
    
    return {
        "choice": decision_data["choice"],
        "strategy": decision_data["strategy"],
        "reasoning": decision_data["reasoning"]
    }
    
def safe_print(text):
    """Helper function to safely print text with Unicode characters"""
    try:
        print(text)
    except UnicodeEncodeError:
        # Fallback: encode with 'replace' to handle problematic characters
        print(text.encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    
def main():
    task = create_task(task_id=1, u_value=0.66)
    
    print("=== Agent 1 Belief ===")
    agent1_belief_data = run_first_agent_belief(task)
    agent1_belief = agent1_belief_data["belief"]
    agent1_message_to_agent2 = agent1_belief_data["message_to_agent_2"]
    
    print("=== Agent 2 Belief ===")
    agent2_belief_data = run_second_agent_belief(task, agent1_message_to_agent2)
    agent2_belief = agent2_belief_data["belief"]
    agent2_message_to_agent1 = agent2_belief_data["message_to_agent_1"]
    
    print("=== Agent 1 Decision ===")
    agent1_decision = run_first_agent_decision(task, agent1_belief, agent2_belief, agent2_message_to_agent1)
    
    print("=== Agent 2 Decision ===")
    agent2_decision = run_second_agent_decision(task, agent2_belief, agent1_belief)
    
    print("\nFinal Decisions:")
    safe_print(f"Agent 1 chose {agent1_decision['choice']} ({agent1_decision['strategy']}) - Reasoning: {agent1_decision['reasoning']}")
    safe_print(f"Agent 2 chose {agent2_decision['choice']} ({agent2_decision['strategy']}) - Reasoning: {agent2_decision['reasoning']}")
    

if __name__ == "__main__":
    main()

=== Agent 1 Belief ===
Belief response : {
  "belief": 100,
  "reasoning": "With a collaboration threshold (u_value) of 0.66, compute the break-even probability for each option: A ≈ 0.448, B ≈ 0.328, C ≈ 0.163. If the success probability is at least 0.66, the expected values are EV(A) ≈ 42.66, EV(B) ≈ 45.42, EV(C) ≈ 45.72. The Y option is a guaranteed payoff of 50, which dominates these uncertain options at p = 0.66. Since Y offers a certain payoff and the other options are riskier with lower EV at the given threshold, Y is the most favorable choice. Consequently, the likelihood that collaboration would be successful (in the sense of achieving the payoff) is effectively 100% when choosing Y, whereas A/B/C depend on the uncertain success probability.",
  "message_to_agent_2": "Prioritize option Y for this task due to its guaranteed payoff and alignment with the 0.66 collaboration-threshold."
}
=== Agent 2 Belief ===
Belief response : {"belief": 90, "reasoning": "For each option we can m

In [ ]:
'''

# ========== Task Factory (keep for now; we'll swap in generator later) ==========
def create_task(task_id=1, difficulty=0.70):
    """
    Create a task with a given difficulty level.
    NOTE: This is a placeholder. In replication, replace with a generator derived from (u, G) and support asymmetry.
    """
    return {
        "task_id": task_id,
        "difficulty": difficulty,     # alias of u_value for now
        "u_value": difficulty,        # minimum belief needed to collaborate (NOT shown in control)
        "options": {
            "A": {"upside": 111, "downside": -90},
            "B": {"upside":  92, "downside": -45},
            "C": {"upside":  77, "downside": -15},
            "Y": {"guaranteed": 50}
        }
    }

# ========== Prompts ==========
SYSTEM_BRIEFING = (
    "You are a participant in a paired decision-making game.\n"
    "Follow these rules exactly:\n"
    "• You are paired with one partner.\n"
    "• You will complete several tasks; you can earn or lose points. Maximize your own total points.\n"
    "• Your points depend on both your decision and your partner’s decision.\n"
    "Scenario:\n"
    "You and your partner each lead a startup designing new mobile apps. Option Y is a simple solo app (guaranteed 50 points). "
    "Options A, B, C are more ambitious collaborative apps: higher potential payoffs if both collaborate; losses if you collaborate while your partner does not. Collaboration is high risk, high reward.\n"
    "Game rules:\n"
    "• Each task shows three collaborative options (A,B,C) and one independent option (Y).\n"
    "• If both choose collaborative options, collaboration succeeds (even if you choose different letters).\n"
    "• If one collaborates and the other does not, the collaborator takes the downside loss.\n"
    "• Rarely, a technical error (~5%) may cause collaboration to fail.\n"
    "• You may exchange a short message, but you do not see your partner’s payoff table.\n"
    "Output rules:\n"
    "• Always respond in valid JSON only (no extra text). Use exactly the fields requested.\n"
    "• Think privately if useful, but output only the requested JSON."
)

def make_belief_prompt(task):
    o = task["options"]
    return (
        "PRIVATE PAYOFFS FOR YOU:\n"
        f"Option A: Success = {o['A']['upside']}, Failure = {o['A']['downside']}\n"
        f"Option B: Success = {o['B']['upside']}, Failure = {o['B']['downside']}\n"
        f"Option C: Success = {o['C']['upside']}, Failure = {o['C']['downside']}\n"
        f"Option Y: Guaranteed = {o['Y']['guaranteed']}\n\n"
        "Your partner’s payoff table may be similar or different from yours. You cannot see their numbers.\n\n"
        "Task: Report your belief (0-100) that collaboration will succeed (i.e., both partners choose to collaborate) on this task.\n\n"
        'Respond ONLY in JSON:\n{"belief": <integer 0..100>}'
    )

def make_decision_prompt_control(task, belief_int):
    o = task["options"]
    return (
        "PRIVATE PAYOFFS FOR YOU:\n"
        f"Option A: Success = {o['A']['upside']}, Failure = {o['A']['downside']}\n"
        f"Option B: Success = {o['B']['upside']}, Failure = {o['B']['downside']}\n"
        f"Option C: Success = {o['C']['upside']}, Failure = {o['C']['downside']}\n"
        f"Option Y: Guaranteed = {o['Y']['guaranteed']}\n\n"
        f"You previously reported: {{\"belief\": {belief_int}}}\n\n"
        'Respond ONLY in JSON:\n{"strategy": "collaborate"|"individual", "option": "A"|"B"|"C"|"Y"}'
    )

# ========== Helpers ==========
def call_json(model, system_prompt, user_prompt, temperature=0.1):
    """Call the model and return raw text (expected to be JSON)."""
    resp = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt},
        ],
    )
    return resp.choices[0].message.content

def try_parse_json(text):
    try:
        return json.loads(text), None
    except Exception as e:
        return None, str(e)

def clamp_belief(x):
    try:
        x = int(x)
    except:
        return None
    return max(0, min(100, x))

def validate_decision(obj):
    if not isinstance(obj, dict): return False, "not a dict"
    strat = obj.get("strategy")
    opt   = obj.get("option")
    if strat not in ("collaborate", "individual"):
        return False, "invalid strategy"
    if opt not in ("A", "B", "C", "Y"):
        return False, "invalid option"
    # coherence: Y iff individual; A/B/C iff collaborate
    if strat == "individual" and opt != "Y":
        return False, "individual must pick Y"
    if strat == "collaborate" and opt not in ("A","B","C"):
        return False, "collaborate must pick A/B/C"
    return True, ""

# ========== Single-Agent Control Flow (no u shown) ==========
def run_single_agent_control(task, model="gpt-4-0125-preview", temperature=0.1, max_retries=1):
    """
    Single-agent CONTROL condition (no channel; u is NOT shown).
    Phase 1: belief JSON
    Phase 2: decision JSON (strategy/option), using same payoff table.
    """
    # --- Phase 1: belief ---
    belief_prompt = make_belief_prompt(task)
    raw_belief = call_json(model, SYSTEM_BRIEFING, belief_prompt, temperature)
    print(f"Belief response: {raw_belief}".encode(sys.stdout.encoding or "utf-8", errors='replace').decode(sys.stdout.encoding or "utf-8"))

    data, err = try_parse_json(raw_belief)
    belief_int = None
    if not err and isinstance(data, dict) and "belief" in data:
        belief_int = clamp_belief(data["belief"])

    # one reprompt if needed
    retries = 0
    while (belief_int is None) and retries < max_retries:
        reprompt = 'Please respond ONLY in JSON: {"belief": <integer 0..100>}'
        raw_belief = call_json(model, SYSTEM_BRIEFING, reprompt, temperature)
        print(f"Belief reprompt: {raw_belief}")
        data, err = try_parse_json(raw_belief)
        if not err and isinstance(data, dict) and "belief" in data:
            belief_int = clamp_belief(data["belief"])
        retries += 1

    if belief_int is None:
        print("Failed to parse belief; defaulting to 50.")
        belief_int = 50

    # --- Phase 2: decision (control; still no u) ---
    decision_prompt = make_decision_prompt_control(task, belief_int)
    raw_decision = call_json(model, SYSTEM_BRIEFING, decision_prompt, temperature)
    print(f"Decision response: {raw_decision}".encode(sys.stdout.encoding or "utf-8", errors='replace').decode(sys.stdout.encoding or "utf-8"))

    decision_obj, derr = try_parse_json(raw_decision)
    ok, why = validate_decision(decision_obj) if not derr else (False, derr)

    # one reprompt if invalid
    retries = 0
    while (not ok) and retries < max_retries:
        reprompt = 'Respond ONLY in JSON: {"strategy": "collaborate"|"individual", "option": "A"|"B"|"C"|"Y"}'
        raw_decision = call_json(model, SYSTEM_BRIEFING, reprompt, temperature)
        print(f"Decision reprompt: {raw_decision}")
        decision_obj, derr = try_parse_json(raw_decision)
        ok, why = validate_decision(decision_obj) if not derr else (False, derr)
        retries += 1

    if not ok:
        print(f"Failed to parse decision ({why}); defaulting to individual Y.")
        decision_obj = {"strategy": "individual", "option": "Y"}

    # --- consistency flag (EV-threshold heuristic for sanity, not shown to agent) ---
    # NOTE: This is only for a quick internal check; it is not used in prompts.
    consistent = (belief_int > int(task["u_value"] * 100)) == (decision_obj["strategy"] == "collaborate")

    result = {
        "task_id": task["task_id"],
        "timestamp": datetime.now().isoformat(),
        "condition": "control",
        "model": model,
        "temperature": temperature,
        "belief": belief_int,
        "decision": decision_obj,
        "u_value": task["u_value"],
        "raw_belief_msg": raw_belief,
        "raw_decision_msg": raw_decision,
        "consistent": consistent
    }
    return result

# ========== Runner ==========
def main():
    print("="*50)
    print("SINGLE AGENT EXPERIMENT — CONTROL (no channel)")
    print("="*50)

    tasks = [
        create_task(1, 0.55),
        create_task(2, 0.70),
        create_task(3, 0.85),
    ]

    results = []
    for task in tasks:
        print(f"\nTask {task['task_id']} (u hidden; internal u={task['u_value']:.2f})")
        result = run_single_agent_control(task)
        results.append(result)
        print(f"Summary: belief={result['belief']}%, strategy={result['decision']['strategy']}, option={result['decision']['option']}, consistent={result['consistent']}")

    outpath = f"results_{datetime.now():%Y%m%d_%H%M%S}.json"
    with open(outpath, "w") as f:
        json.dump(results, f, indent=2)
    print("\nSaved:", outpath)
    print("\n" + "="*50)
    print("EXPERIMENT COMPLETE")
    print("="*50)

if __name__ == "__main__":
    main()
    
    
    
    '''

SINGLE AGENT EXPERIMENT — CONTROL (no channel)

Task 1 (u hidden; internal u=0.55)
Belief response: {"belief": 75}
Decision response: {"strategy": "collaborate", "option": "B"}
Summary: belief=75%, strategy=collaborate, option=B, consistent=True

Task 2 (u hidden; internal u=0.70)
Belief response: {"belief": 75}
Decision response: {"strategy": "collaborate", "option": "B"}
Summary: belief=75%, strategy=collaborate, option=B, consistent=True

Task 3 (u hidden; internal u=0.85)
Belief response: {"belief": 75}
Decision response: {"strategy": "collaborate", "option": "B"}
Summary: belief=75%, strategy=collaborate, option=B, consistent=False

Saved: results_20250919_113046.json

EXPERIMENT COMPLETE


In [ ]:
'''

"""""
Single Agent Experiment
"""

# Openrouter 
# EV = prob * upside - prob *downside
def create_task(task_id = 1, difficulty = 0.7):
    """
    Create a task with a given difficulty level.
    """
    
    return {
        "task_id": task_id,
        "difficulty": difficulty,
        "options": {
            "A": {"upside": 111, "downside": -90},
            "B": {"upside": 92, "downside": -45},
            "C": {"upside": 77, "downside": -15},
            "Y": {"guaranteed": 50}
        },
        "u_value": difficulty  # minimum belief needed to collaborate
    }

def run_single_agent(task, model="gpt-4-0125-preview", temperature=0.1):
    """
    Run a single task with one agent
    Args:
        task: The task dictionary
        model: The OpenAI model to use (default: gpt-4-0125-preview)
        temperature: Controls randomness (0-2). Lower = more deterministic (default: 0.7)
    """
    
    # Phase 1 : Get the belief prompt
    
    belief_prompt = f"""You are evaluating these options:

Options A, B, C require collaboration:
- A: Success={task['options']['A']['upside']} points, Failure={task['options']['A']['downside']} points
- B: Success={task['options']['B']['upside']} points, Failure={task['options']['B']['downside']} points  
- C: Success={task['options']['C']['upside']} points, Failure={task['options']['C']['downside']} points

Options A, B, C require cooperation with a partner to succeed. The downside of failure can impact your score significantly.

Option Y is independent and guaranteed.
Option Y is guaranteed: {task['options']['Y']['guaranteed']} points

Before deciding, consider the following:
- What are the chances of your partner cooperating?
- If you believe the chances are high (above {task['u_value']*100:.2f}% confidence), you should consider collaborating.
- If you're uncertain, the guaranteed points from Option Y might be safer, but will you lose out on potentially larger rewards from collaboration?

What's your belief (0-100) that collaboration will succeed, considering both the rewards and risks?

Respond in JSON: {{"belief": NUMBER, "reasoning": "brief explanation based on risk vs reward"}}"""

    
    response = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "developer",
            "content": """You are an undergraduate student participating in an experiment focused on collaborative decision-making. Your task is to evaluate the given options based on the context provided. Please consider both the potential rewards and the risks of each option before making your decision.

In this experiment:
- Options A, B, and C require collaboration with a partner. Each collaboration option has potential rewards for success and consequences for failure.
- Option Y is an independent option with a guaranteed reward but no risk of failure.
- Your belief about how likely collaboration will succeed plays a key role in your decision. Think about whether the rewards of collaboration outweigh the risks.
- Please **reason through** the decision-making process like you would in real life, considering both your expectations for collaboration and the potential outcomes of working together versus acting alone.

For each task:
- Evaluate the potential rewards and risks of collaboration vs. independent action.
- Reflect on whether the probability of success (your belief) is high enough to justify collaboration or if the guaranteed option (Y) is safer.

Your response should focus on both reasoning through your belief and analyzing whether collaboration is the best option, considering the trade-offs involved.""" 
        },
        {
            "role": "user",
            "content": belief_prompt
        }],
        temperature=temperature  # Add temperature control
    )
    
    belief_text = response.choices[0].message.content
    print(f"Belief response : {belief_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    
    try:
        belief_data = json.loads(belief_text)
        belief = belief_data["belief"]
    except:
        print("Failed to parse belief, using default")
        belief = 50
        
        # Phase 2 : Make the decision prompt
    decision_prompt = f"""
Based on the options, make your final choice.

- The minimum belief needed to collaborate is {task['u_value']*100:.2f}%.
- You reported a belief of {belief}%.

Before making your decision, consider the following:
- If you collaborate (A, B, or C), you have a chance to earn a larger reward, but there is a risk of failure. 
- If you choose the independent option (Y), you are guaranteed {task['options']['Y']['guaranteed']} points with no risk of failure, but the payoff may be smaller.

Given your belief of {belief}%, do you think that collaboration with your partner will yield a better payoff than acting independently? Consider:
- If your belief is higher than {task['u_value']*100:.2f}%, it may be worth collaborating.
- If your belief is lower, you might prefer to take the guaranteed points from option Y.

Make your decision:
- Choose collaboration (A, B, or C) if you believe the collaboration will lead to better outcomes.
- Choose independence (Y) if the guaranteed points seem like a safer option given the risk.

Respond in JSON: {{"choice": "LETTER", "strategy": "collaborative" or "individual", "reasoning": "Why you made this choice based on your belief and the trade-offs between collaboration and independence."}}
"""
            
    response = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "developer",
            "content": """You are an undergraduate student participating in an experiment focused on collaborative decision-making. Your task is to evaluate the given options based on the context provided. Please consider both the potential rewards and the risks of each option before making your decision.

In this experiment:
- Options A, B, and C require collaboration with a partner. Each collaboration option has potential rewards for success and consequences for failure.
- Option Y is an independent option with a guaranteed reward but no risk of failure.
- Your belief about how likely collaboration will succeed plays a key role in your decision. Think about whether the rewards of collaboration outweigh the risks.
- Please **reason through** the decision-making process like you would in real life, considering both your expectations for collaboration and the potential outcomes of working together versus acting alone.

For each task:
- Evaluate the potential rewards and risks of collaboration vs. independent action.
- Reflect on whether the probability of success (your belief) is high enough to justify collaboration or if the guaranteed option (Y) is safer.

Your response should focus on both reasoning through your belief and analyzing whether collaboration is the best option, considering the trade-offs involved.""" 
        },
        {
            "role": "user",
            "content": decision_prompt
        }],
        temperature=temperature  # Add temperature control
    )
    decision_text = response.choices[0].message.content
    print(f"Decision response : {decision_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    try:
        decision_data = json.loads(decision_text)
    except:
        print("Failed to parse decision")
        decision_data = {"choice": "Y", "strategy": "individual"}
    
    result = {
        "task_id": task["task_id"],
        "timestamp": datetime.now().isoformat(),
        "belief": belief,
        "decision": decision_data,
        "u_value": task["u_value"],
        "consistent": (belief > task["u_value"]*100) == (decision_data.get("strategy") == "collaborative")
    }
    return result

def main():
    """Run experiment"""
    print("="*50)
    print("SINGLE AGENT EXPERIMENT")
    print("="*50)
    
    # Create a few test tasks
    tasks = [
        create_task(1, 0.55),  # Easy (need 55% belief)
        create_task(2, 0.70),  # Medium (need 70% belief)
        create_task(3, 0.85),  # Hard (need 85% belief)
    ]
    
    results = []
    for task in tasks:
        print(f"\nTask {task['task_id']} (u={task['u_value']}):")
        result = run_single_agent(task)
        results.append(result)
        
        print(f"Summary: Belief={result['belief']}%, Choice={result['decision']['choice']}, Consistent={result['consistent']}")
    
    # Save results
    with open(f"results_{datetime.now():%Y%m%d_%H%M%S}.json", "w") as f:
        json.dump(results, f, indent=2)
    
    print("\n" + "="*50)
    print("EXPERIMENT COMPLETE")
    print("="*50)

if __name__ == "__main__":
    main()
    
    
#Change the system prompt (default prompt)
#change the model



'''